### Import packages

In [1]:
import os
import json
import pandas as pd
import numpy as np
import sqlite3
import json
import copy
import re
import operator
import math

### Import CCSR data

In [73]:
# ccsr = pd.read_excel("disease_hierarchy/CCSR_for_ICD10/CCSR.xlsx", sheet_name='DX_to_CCSR_Mapping', skiprows=1)

In [2]:
ccsr = pd.read_csv("disease_hierarchy/CCSR_for_ICD10/DXCCSR_v2021-1.CSV")

In [3]:
ccsr.columns = [x.replace("'",'') for x in ccsr.columns]

In [4]:
ccsr = ccsr[ccsr.columns[:6,].values]

In [5]:
for n in [0,2,4]:
    ccsr.iloc[:,n] = [x.replace("'",'') for x in ccsr.iloc[:,n].to_list()]

In [6]:
ccsr.head()

,ICD-10-CM CODE,ICD-10-CM CODE DESCRIPTION,Default CCSR CATEGORY IP,Default CCSR CATEGORY DESCRIPTION IP,Default CCSR CATEGORY OP,Default CCSR CATEGORY DESCRIPTION OP
0,A000,"Cholera due to Vibrio cholerae 01, biovar chol...",DIG001,Intestinal infection,DIG001,Intestinal infection
1,A001,"Cholera due to Vibrio cholerae 01, biovar eltor",DIG001,Intestinal infection,DIG001,Intestinal infection
2,A009,"Cholera, unspecified",DIG001,Intestinal infection,DIG001,Intestinal infection
3,A0100,"Typhoid fever, unspecified",DIG001,Intestinal infection,DIG001,Intestinal infection
4,A0101,Typhoid meningitis,NVS001,Meningitis,NVS001,Meningitis


In [57]:
dh = pd.read_excel("disease_hierarchy/disease_hierarchy.xlsx")

In [66]:
merged = ccsr.merge(dh, how='left', left_on = 'Default CCSR CATEGORY DESCRIPTION IP', right_on ='Level 3')

In [74]:
#merged.to_csv("ccsr_manual_hierarchy.csv", index=False)

##### hierarchy was manually revised

In [2]:
ccsr = pd.read_csv("disease_hierarchy/ccsr_manual_hierarchy_revised.csv")

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
ccsr['Level 2'].unique()

array(['Coronary artery disease', 'Heart failure',
       'Other cardiovascular disorders', 'Stroke', 'No specific choices',
       'Diabetes', 'Other endocrine disorders', 'Liver diseases',
       'Other gastrointestinal disorders', 'Chronic kidney disease',
       'Other genitourinary disorders', 'HIV/AIDS', 'Other infections',
       'Other injuries and external causes', 'Other mental disorders',
       'Substance use disorders', 'Other musculoskeletal disorders',
       'Rheumatologic disorders', 'Brain cancer', 'Breast cancer',
       'Colorectal cancer', 'Other neoplasms', 'Hematologic malignancies',
       'Prostate cancer', 'Lung cancer', 'Multiple sclerosis',
       'Neurodegenerative diseases', 'Other neurologic disorders',
       'Neurodenerative diseases', 'COPD', 'Other respiratory disorders',
       nan], dtype=object)

In [4]:
ccsr['Level 1'].unique()

array(['Cardiovascular diseases',
       'Chromosomal, congenital, and perinatal disorders',
       'Endocrine diseases', 'Gastrointestinal diseases',
       'Genitourinary disorders', 'Infections',
       'Injuries and external causes',
       'Mental, behavioral, and substance use disorders',
       'Musculoskeletal diseases', 'Neoplasms', 'Nervous system diseases',
       'Nonspecific', 'Other noncommunicable diseases',
       'Pregnancy and childbirth', 'Respiratory diseases', nan],
      dtype=object)

In [5]:
ccsr.columns

Index(['ICD-10-CM CODE', 'ICD-10-CM CODE DESCRIPTION',
       'Default CCSR CATEGORY IP', 'Default CCSR CATEGORY DESCRIPTION IP',
       'Default CCSR CATEGORY OP', 'Default CCSR CATEGORY DESCRIPTION OP',
       'Level 3', 'Level 2', 'Level 1'],
      dtype='object')

In [2]:
ccsr = pd.read_excel('Manual_Review/disease_hierarchy_11.30.2020.xlsx')

### Import SNOMED to ICD data

In [5]:
conn = sqlite3.connect("SNOMED_to_ICD10.db")
snomed = pd.read_sql_query("SELECT * FROM map_tb", conn)
conn.close()

### Import ATHENA data

In [12]:
conn = sqlite3.connect("ATHENA.db")
athena = pd.read_sql_query("SELECT * FROM concept_tb WHERE domain_id = 'Condition'", conn)
conn.close()

### Import NER output

In [14]:
with open('Manual_Review/round 2/ner_disease_ner_clinical_icd10.json') as f:
     r_icd = json.load(f)

In [14]:
with open('Manual_Review/round 2/ner_disease_ner_clinical_snomed.json') as f:
     r_snomed = json.load(f)

In [15]:
with open('Manual_Review/round 2/ner_disease_ner_healthcare_athena.json') as f:
     r_athena = json.load(f)

### Define Functions

In [15]:
def GetTopICD(lofd):
    results = []
    for d in lofd:
        dic = copy.deepcopy(d)
        for n in range(0, len(d['entity'])):
            dic['entity'][n]['icd_code'] = dic['entity'][n]['all_k_results'].split(':::')[0]
        results.append(dic)
    return results

In [16]:
def GetTopSnomed(lofd):
    results = []
    for d in lofd:
        dic = copy.deepcopy(d)
        for n in range(0, len(d['entity'])):
            dic['entity'][n]['concept_code'] = dic['entity'][n]['all_k_results'].split(':::')[0]
        results.append(dic)
    return results

In [17]:
def SnomedToICD(df):
    asdf = copy.deepcopy(df)
    asdf['concept_code'] = asdf['concept_code'].astype(int)
    merge = asdf.merge(snomed, how='left', left_on='concept_code', right_on='referencedComponentId')
    # FIND WAY TO SELECT DEFAULT SNOMED
    merge = merge[merge['mapGroup'] == 1]
    merge['mapTarget'] = [x.replace('.','') if x != None else x for x in merge['mapTarget']]
    merge = merge.dropna(subset=['mapTarget'])
    return merge

In [18]:
def SnomedToCategoryScores(lofd, distance):
    results = []
    for d in lofd:
        dic = copy.deepcopy(d)
        df = pd.DataFrame(dic['entity'])
        df_sno_icd = SnomedToICD(df)
        df_sno_icd_ccsr = df_sno_icd.merge(ccsr, how='left', left_on='mapTarget', right_on='ICD-10-CM CODE')
        dic['merge'] = df_sno_icd_ccsr
        dic['scores_level_2'] = df_sno_icd_ccsr.groupby(['Level 2'])[distance].sum().to_dict()
        dic['scores_level_1'] = df_sno_icd_ccsr.groupby(['Level 1'])[distance].sum().to_dict()
        dic['level_2'] = max(dic['scores_level_2'].items(), key=operator.itemgetter(1))[0]
        dic['level_1'] = max(dic['scores_level_1'].items(), key=operator.itemgetter(1))[0]
        results.append(dic)
    return results

In [19]:
def MergeAthena(lofd):
    results = []
    for d in lofd:
        dic = copy.deepcopy(d)
        if len(dic['entity']) == 0:
            dic['merge'] = {}
            results.append(dic)
        else:
            df = pd.DataFrame(dic['entity'])
            df['all_k_results'] = df['all_k_results'].astype(int)
            merge = df.merge(athena, how='left', left_on='all_k_results', right_on='concept_id')
            dic['merge'] = merge.to_dict()
            results.append(dic)
    return results

In [20]:
def AthenaToCategoryScores(lofd, distance):
    results = []
    for d in lofd:
        dic = copy.deepcopy(d)
        if len(dic['merge']) == 0:
            dic['scores_level_2'] = np.nan
            dic['scores_level_1'] = np.nan
            dic['level_2'] = np.nan
            dic['level_1'] = np.nan
            results.append(dic)
        else:
            df = pd.DataFrame(dic['merge'])
            df = df.dropna(subset=['concept_code'])
            if df['vocabulary_id'].str.contains('ICD10CM').any():
                if df['vocabulary_id'].str.contains('SNOMED').any():

                    #ICD10 susbet
                    df_icd = df[df['vocabulary_id'] == 'ICD10CM']
                    df_icd_ccsr = df_icd.merge(ccsr, how='left', left_on='concept_code', right_on='ICD-10-CM CODE')

                    #SNOMED subset
                    df_sno = df[df['vocabulary_id'] == 'SNOMED']
                    df_sno_icd = SnomedToICD(df_sno)
                    df_sno_icd_ccsr = df_sno_icd.merge(ccsr, how='left', left_on='mapTarget', right_on='ICD-10-CM CODE')

                    #Row-bind subsets
                    df_sno_icd_ccsr = df_sno_icd_ccsr[['Level 3','Level 2','Level 1','inv_distance','exp_distance']]
                    df_icd_ccsr = df_icd_ccsr[['Level 3','Level 2','Level 1','inv_distance','exp_distance']]
                    rbind = pd.concat([df_sno_icd_ccsr,df_icd_ccsr])
                    dic['scores_level_2'] = rbind.groupby(['Level 2'])[distance].sum().to_dict()
                    dic['scores_level_1'] = rbind.groupby(['Level 1'])[distance].sum().to_dict()
                    if len(dic['scores_level_2']) == 0:
                        dic['level_2'] = np.nan
                    else:
                        dic['level_2'] = max(dic['scores_level_2'].items(), key=operator.itemgetter(1))[0]
                    if len(dic['scores_level_1']) == 0:
                        dic['level_1'] = np.nan
                    else:
                        dic['level_1'] = max(dic['scores_level_1'].items(), key=operator.itemgetter(1))[0]

                else:
                    df_icd_ccsr = df.merge(ccsr, how='left', left_on='concept_code', right_on='ICD-10-CM CODE')
                    dic['scores_level_2'] = df_icd_ccsr.groupby(['Level 2'])[distance].sum().to_dict()
                    dic['scores_level_1'] = df_icd_ccsr.groupby(['Level 1'])[distance].sum().to_dict()
                    if len(dic['scores_level_2']) == 0:
                        dic['level_2'] = np.nan
                    else:
                        dic['level_2'] = max(dic['scores_level_2'].items(), key=operator.itemgetter(1))[0]
                    if len(dic['scores_level_1']) == 0:
                        dic['level_1'] = np.nan
                    else:
                        dic['level_1'] = max(dic['scores_level_1'].items(), key=operator.itemgetter(1))[0]

            else:
                df_sno_icd = SnomedToICD(df)
                df_sno_icd_ccsr = df_sno_icd.merge(ccsr, how='left', left_on='mapTarget', right_on='ICD-10-CM CODE')
                dic['scores_level_2'] = df_sno_icd_ccsr.groupby(['Level 2'])[distance].sum().to_dict()
                dic['scores_level_1'] = df_sno_icd_ccsr.groupby(['Level 1'])[distance].sum().to_dict()
                if len(dic['scores_level_2']) == 0:
                    dic['level_2'] = np.nan
                else:
                    dic['level_2'] = max(dic['scores_level_2'].items(), key=operator.itemgetter(1))[0]
                if len(dic['scores_level_1']) == 0:
                    dic['level_1'] = np.nan
                else:
                    dic['level_1'] = max(dic['scores_level_1'].items(), key=operator.itemgetter(1))[0]



            results.append(dic)
    return results

In [21]:
def ScaleDistance(lofd):
    results = []
    for d in lofd:
        dic = copy.deepcopy(d)
        for  n in range(0, len(d['entity'])):
            dic['entity'][n]['inv_distance'] = [1/x if x > 0 else 100 for x in [float(dic['entity'][n]['distance'])]][0]
            dic['entity'][n]['exp_distance'] = [math.exp(-x) for x in [float(dic['entity'][n]['distance'])]][0]
        results.append(dic)
    return results

In [126]:
# def CalculateSentenceScore(lofd):
#     '''
#     calculates inverse of sentence position/total number of sentences
#     '''
#     results = []
#     for d in lofd:
#         dic = copy.deepcopy(d)
#         num_sent = dic['num_sentences']
#         for  n in range(0, len(d['entity'])):
#             #make sentence position 1-based
#             sent = int(dic['entity'][n]['sentence']) + 1
#             dic['entity'][n]['sent_score'] = 1/(sent/num_sent)
#         results.append(dic)
#     return results

In [22]:
def GetCategoryScoresICD(lofd, distance):
    results = []
    for d in lofd:
        dic = copy.deepcopy(d)
        if len(dic['entity']) == 0:
            dic['scores_level_2'] = np.nan
            dic['scores_level_1'] = np.nan
            dic['level_2'] = np.nan
            dic['level_1'] = np.nan
            results.append(dic)
        else:
            df = pd.DataFrame(dic['entity'])
            merge = df.merge(ccsr, how='left', left_on='icd_code', right_on='ICD-10-CM CODE')
            dic['merge'] = merge.to_dict()
            dic['scores_level_2'] = merge.groupby(['Level 2'])[distance].sum().to_dict()
            dic['scores_level_1'] = merge.groupby(['Level 1'])[distance].sum().to_dict()
            if len(dic['scores_level_2']) == 0:
                dic['level_2'] = np.nan
            else:
                dic['level_2'] = max(dic['scores_level_2'].items(), key=operator.itemgetter(1))[0]
            if len(dic['scores_level_1']) == 0:
                dic['level_1'] = np.nan
            else:
                dic['level_1'] = max(dic['scores_level_1'].items(), key=operator.itemgetter(1))[0]
            results.append(dic)
    return results

#### ICD pipeline

In [48]:
df = pd.DataFrame(r_icd)

In [85]:
import nltk
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")
def RemoveStopwords(s):
    return ' '.join([word for word in s.split() if word not in cachedStopWords])

In [86]:
def GetNER(lofd):
    df = pd.DataFrame(lofd)
    r = []
    for chunk in df['ner_chunk']:
        if len(chunk) < 1:
            r.append([])
        else:
            l = []
            for res in chunk:
                l.append(RemoveStopwords(res['result'].lower()))
            r.append(l)
    return r

In [87]:
df['ner'] = GetNER(df)

In [88]:
df['ner'][19]

['retinopathy prematurity',
 'eye cancer right eye',
 'retinoblastoma',
 'cancer',
 'progressive decrease',
 'retinoblastoma',
 'agradecería']

In [89]:
df['fund_description'][19]

' ENGLISH    My name is Dahiana Clavijo and I was born with a condition called retinopathy of prematurity, because I was born five months of gestation. At nine years old, I was detected an eye cancer in the right eye called retinoblastoma, which causes my eye to be removed. A few months ago, when I went to an ophthalmologic review, I am informed that the cancer has returned but that my body blocked it by calcifying the back of my left eye and causing its progressive decrease; The doctor offers me an alternative to surgery, because it is quite expensive and I do not have the resources or social security to pay for it; The alternative is to buy two prostheses so that my eyes look the same size and color, thus increasing my self-confidence. I thank you in advance for the support you want to give me and although I know it is not easy, I hope you can donate. thank you very much for your attention.  ESPAÑOL  Mi nombre es Dahiana Clavijo y nací con una afección llamada retinopatía del prematu

In [44]:
df = pd.DataFrame(r)

In [45]:
df

,url,fund_description,num_sentences,ner_chunk,chunks,new_doc,entity,scores_level_2,scores_level_1,level_2,level_1,merge
0,https://www.gofundme.com/f/JosiahAnakinLopez,We will be having my sons open heart surgery i...,2,[],[],,[],NaN,NaN,NaN,NaN,NaN
1,https://www.gofundme.com/f/Kayataskidsclinic,THE STORY: Welcome! My name is Porkpa Tokpa ...,49,"[{'result': 'infections', 'sentence': 18}, {'r...","[infections, fevers, clinic's septic tank]",infections and fevers and clinic's septic tank,"[{'sentence': '0', 'all_k_distances': '0.0000:...","{'HIV/AIDS': 1.0, 'No specific choices': 0.667...","{'Infections': 1.0, 'Musculoskeletal diseases'...",HIV/AIDS,Infections,"{'sentence': {0: '0', 1: '0', 2: '0'}, 'all_k_..."
2,https://www.gofundme.com/f/lindsey-and-adam-di...,"Lindsey and Adam met in high school, supported...",9,"[{'result': 'acute neurological dysfunction', ...",[acute neurological dysfunction],acute neurological dysfunction,"[{'sentence': '0', 'all_k_distances': '1.0869:...",{},{},NaN,NaN,"{'sentence': {0: '0'}, 'all_k_distances': {0: ..."
3,https://www.gofundme.com/f/ruths-skin-removal-...,￼My name is Ruth I weighed 380lbs plus. I had ...,24,"[{'result': 'sleep apnea', 'sentence': 1}, {'r...","[sleep apnea, borderline diabetic, it.The pain...",sleep apnea and borderline diabetic and it.The...,"[{'sentence': '0', 'all_k_distances': '0.2354:...",{'Coronary artery disease': 0.6930175191174569...,{'Cardiovascular diseases': 0.6930175191174569...,No specific choices,Nonspecific,"{'sentence': {0: '0', 1: '0', 2: '0', 3: '0', ..."
4,https://www.gofundme.com/f/billbrownlv,Most people celebrate the holidays by looking ...,37,"[{'result': 'the pressure on his brain', 'sent...","[pressure on brain, basically BORG, stroke, pr...",pressure on brain and basically BORG and strok...,"[{'sentence': '0', 'all_k_distances': '0.9995:...",{'Stroke': 0.3680634268846233},{'Cardiovascular diseases': 0.3680634268846233},Stroke,Cardiovascular diseases,"{'sentence': {0: '0'}, 'all_k_distances': {0: ..."
5,https://www.gofundme.com/f/angel-sarah-forever...,"Sarah was a bright and determined girl, seekin...",5,"[{'result': 'Acute Lymphoblastic Leukemia', 's...",[Acute Lymphoblastic Leukemia],Acute Lymphoblastic Leukemia,"[{'sentence': '0', 'all_k_distances': '0.3392:...",{'Hematologic malignancies': 0.712339966848073},{'Neoplasms': 0.712339966848073},Hematologic malignancies,Neoplasms,"{'sentence': {0: '0'}, 'all_k_distances': {0: ..."
6,https://www.gofundme.com/f/SavingTyisha,"My beautiful sister, Tyisha Prescott, who was ...",37,"[{'result': 'Stage 4 Undifferentiated Cancer',...","[Stage 4 Undifferentiated Cancer, mass between...",Stage 4 Undifferentiated Cancer and mass betwe...,"[{'sentence': '0', 'all_k_distances': '0.9471:...",{'Coronary artery disease': 0.6703200460356393...,{'Cardiovascular diseases': 0.6703200460356393...,Other neoplasms,Neoplasms,"{'sentence': {0: '0', 1: '0', 2: '0', 3: '0', ..."
7,https://www.gofundme.com/f/xm4c4cq7q,"""Your life can change in an instant."" We've al...",20,"[{'result': 'badly injured', 'sentence': 7}, {...","[badly injured, unresponsive]",badly injured and unresponsive,"[{'sentence': '0', 'all_k_distances': '1.3075:...",{'No specific choices': 0.12176041214231789},{'Other noncommunicable diseases': 0.121760412...,No specific choices,Other noncommunicable diseases,"{'sentence': {0: '0', 1: '0'}, 'all_k_distance..."
8,https://www.gofundme.com/f/cheyennejensen,There are so many people who love Cheyenne Hop...,19,"[{'result': 'AML Leukemia', 'sentence': 2}, {'...","[AML Leukemia, cancer]",AML Leukemia and cancer,"[{'sentence': '0', 'all_k_distances': '0.9312:...",{'Other neoplasms': 0.9977068184972714},{'Neoplasms': 0.9977068184972714},Other neoplasms,Neoplasms,"{'sentence': {0: '0', 1: '0'}, 'all_k_distance..."
9,https://www.gofundme.com/f/22iq896m5c,Iam raising money to pay off my medical expens...,1,[],[],,[],NaN,NaN,NaN,NaN,NaN


In [43]:
r = GetCategoryScoresICD(GetTopICD(ScaleDistance(r_icd)), 'exp_distance')
# lofd = ScaleDistance(GetTopICD(r_icd))
# d = lofd[2]
# df = pd.DataFrame(d['entity'])
# merge = df.merge(ccsr, how='left', left_on='icd_code', right_on='ICD-10-CM CODE')
# d['scores_level_2'] = merge.groupby(['Level 2'])['exp_distance'].sum().to_dict()
# d['scores_level_1'] = merge.groupby(['Level 1'])['exp_distance'].sum().to_dict()
# d

In [94]:
df = pd.DataFrame(r)
df = df[['url','level_1']]
df = df.fillna('Nonspecific')
df.to_csv('Manual_Review/round 2/icd10_results.csv')

In [124]:
df = pd.DataFrame(r)

In [127]:
pd.DataFrame(df[df['url'] == 'https://www.gofundme.com/f/ruths-skin-removal-surgery']['merge'][3])

,sentence,all_k_distances,resolved_text,target_text,all_k_confidences,distance,all_k_resolutions,confidence,chunk,all_k_results,token,icd_code,inv_distance,exp_distance,ICD-10-CM CODE,ICD-10-CM CODE DESCRIPTION,Level 3,Level 2,Level 1
0,0,0.2354:::0.2392:::0.4844:::0.4850:::0.6236:::0...,"Sleep apnea, unspecified",sleep apnea,0.0839:::0.0836:::0.0654:::0.0654:::0.0569:::0...,0.2354,"Sleep apnea, unspecified:::Other sleep apnea::...",0.0839,0,G4730:::G4739:::G4731:::R092:::P283:::G4737:::...,sleep apnea,G4730,4.248088,0.790255,G4730,"Sleep apnea, unspecified",Sleep wake disorders,Other neurologic disorders,Nervous system diseases
1,0,0.6295:::0.9365:::1.0293:::1.0695:::1.0695:::1...,Prediabetes,borderline diabetic,0.0795:::0.0585:::0.0533:::0.0512:::0.0512:::0...,0.6295,Prediabetes:::Borderline personality disorder:...,0.0795,1,R7303:::F603:::A303:::A304:::A302:::E71121:::E...,borderline diabetic,R7303,1.588562,0.532858,R7303,Prediabetes,Abnormal findings without diagnosis,No specific choices,Nonspecific
2,0,0.3667:::0.3738:::0.4356:::0.4426:::0.4599:::0...,Precordial pain,it.The pain,0.0477:::0.0474:::0.0445:::0.0442:::0.0435:::0...,0.3667,"Precordial pain:::Pain, unspecified:::Cervical...",0.0477,2,R072:::R52:::M542:::G8922:::G8912:::N644:::R14...,it.The pain,R072,2.727025,0.693018,R072,Precordial pain,Nonspecific chest pain,Coronary artery disease,Cardiovascular diseases
3,0,0.8906:::0.9239:::0.9440:::0.9641:::0.9660:::0...,"Pain, unspecified",physical pain,0.0456:::0.0441:::0.0432:::0.0423:::0.0423:::0...,0.8906,"Pain, unspecified:::Age-related physical debil...",0.0456,3,R52:::R54:::G8922:::M542:::G8912:::R072:::R103...,physical pain,R52,1.122839,0.410409,R52,"Pain, unspecified",Other general signs and symptoms,No specific choices,Nonspecific
4,0,0.9552:::0.9614:::0.9834:::0.9952:::1.0155:::1...,"Pain, unspecified",emotional pain,0.0449:::0.0447:::0.0437:::0.0432:::0.0423:::0...,0.9552,"Pain, unspecified:::Emotional lability:::Chron...",0.0449,4,R52:::R4586:::G8922:::G8912:::M542:::R1031:::R...,emotional pain,R52,1.046901,0.384735,R52,"Pain, unspecified",Other general signs and symptoms,No specific choices,Nonspecific
5,0,0.5398:::0.7773:::1.1480:::1.1891:::1.2313:::1...,Underweight,my extreme weight loss,0.0817:::0.0644:::0.0445:::0.0427:::0.0409:::0...,0.5398,Underweight:::Abnormal weight loss:::Bariatric...,0.0817,5,R636:::R634:::O99845:::R635:::O99842:::Z595:::...,my extreme weight loss,R636,1.852538,0.582865,R636,Underweight,Other general signs and symptoms,No specific choices,Nonspecific
6,0,0.9205:::0.9503:::0.9519:::0.9660:::0.9872:::1...,Estrogen excess,excess skin,0.0470:::0.0456:::0.0455:::0.0449:::0.0440:::0...,0.9205,Estrogen excess:::Cutaneous mastocytosis:::And...,0.0470,6,E280:::D4701:::E281:::H5112:::E875:::R200:::I7...,excess skin,E280,1.086366,0.398320,E280,Estrogen excess,Other specified and unspecified endocrine diso...,Other endocrine disorders,Endocrine diseases
7,0,0.3732:::0.6706:::1.0615:::1.1091:::1.1145:::1...,Underweight,extreme weight loss,0.0895:::0.0665:::0.0450:::0.0429:::0.0427:::0...,0.3732,Underweight:::Abnormal weight loss:::Bariatric...,0.0895,7,R636:::R634:::O99845:::R635:::Z595:::R29890:::...,extreme weight loss,R636,2.679528,0.688528,R636,Underweight,Other general signs and symptoms,No specific choices,Nonspecific
8,0,0.5481:::0.7347:::0.7394:::1.0962:::1.2554:::1...,Parosmia,changes,0.0858:::0.0712:::0.0709:::0.0496:::0.0423:::0...,0.5481,Parosmia:::Unspecified skin changes:::Other sk...,0.0858,8,R431:::R239:::R238:::R234:::M9120:::F0390:::H0...,changes,R431,1.824485,0.578047,R431,Parosmia,Nervous system signs and symptoms,No specific choices,Nonspecific


#### Athena pipeline

In [108]:
r = AthenaToCategoryScores(MergeAthena(ScaleDistance(r_athena)), 'inv_distance')

In [109]:
df = pd.DataFrame(r)
df = df[['url','level_1']]
df = df.fillna('Nonspecific')
df.to_csv('Manual_Review/round 2/athena_results_inv.csv')

#### SNOMED pipeline

In [32]:
#r = SnomedToCategoryScores(ScaleDistance(GetTopSnomed(r_snomed)), 'exp_distance')

# Spacy NEr

In [46]:
df = pd.read_csv('Comparing algorithms/round 2 campaign terms/round_2_ner_disease_cui_code.csv')

In [47]:
df = df[df['sab'] == 'ICD10CM']

In [48]:
def GetIdxMax(df):
    idx_max = []
    for url in df['url'].unique():
        d = copy.deepcopy(df)
        d = d[d['url'] == url]
        for text in d['text'].unique():
            f = copy.deepcopy(d)
            f = f[f['text'] == text]
            idx_max.append(f['score'].idxmax())
    return idx_max

In [49]:
a = df[df['url'] == 'https://www.gofundme.com/f/wesley-needs-a-liver-transplant']

In [50]:
l = GetIdxMax(df)

In [51]:
df.loc[l].head()

,url,text,label,start_char,end_char,score,cui,lang,ts,lui,sab,tty,code
4,https://www.gofundme.com/f/help-piper-afford-a...,arthritis,DISEASE,0,9,1.0,C0003864,ENG,P,L0003864,ICD10CM,ET,M19
15,https://www.gofundme.com/f/tangopolly,arthritis,DISEASE,110,119,1.0,C0003864,ENG,P,L0003864,ICD10CM,ET,M19
6037,https://www.gofundme.com/f/tangopolly,COPD,DISEASE,154,158,1.0,C0024117,ENG,P,L0498186,ICD10CM,ET,J44
6240,https://www.gofundme.com/f/tangopolly,respiratory failure,DISEASE,0,19,1.0,C1145670,ENG,S,L0834696,ICD10CM,AB,J96
6318,https://www.gofundme.com/f/tangopolly,breast cancer,DISEASE,34,47,1.0,C0006142,ENG,P,L0531189,ICD10CM,AB,C50


In [52]:
ccsr['ICD-10-CM CODE'] = [x.replace('.','')[:3] for x in ccsr['ICD-10-CM CODE']]

In [53]:
ccsr = ccsr.drop_duplicates('ICD-10-CM CODE')

In [63]:
m = df.loc[l].merge(ccsr, how='left', left_on='code', right_on='ICD-10-CM CODE')

In [55]:
m.head()

,url,text,label,start_char,end_char,score,cui,lang,ts,lui,sab,tty,code,ICD-10-CM CODE,ICD-10-CM CODE DESCRIPTION,Level 3,Level 2,Level 1
0,https://www.gofundme.com/f/help-piper-afford-a...,arthritis,DISEASE,0,9,1.0,C0003864,ENG,P,L0003864,ICD10CM,ET,M19,M19,"Primary osteoarthritis, right shoulder",Osteoarthritis,Other musculoskeletal disorders,Musculoskeletal diseases
1,https://www.gofundme.com/f/tangopolly,arthritis,DISEASE,110,119,1.0,C0003864,ENG,P,L0003864,ICD10CM,ET,M19,M19,"Primary osteoarthritis, right shoulder",Osteoarthritis,Other musculoskeletal disorders,Musculoskeletal diseases
2,https://www.gofundme.com/f/tangopolly,COPD,DISEASE,154,158,1.0,C0024117,ENG,P,L0498186,ICD10CM,ET,J44,J44,Chronic obstructive pulmonary disease with acu...,Chronic obstructive pulmonary disease and bron...,COPD,Respiratory diseases
3,https://www.gofundme.com/f/tangopolly,respiratory failure,DISEASE,0,19,1.0,C1145670,ENG,S,L0834696,ICD10CM,AB,J96,J96,"Acute respiratory failure, unspecified whether...",Respiratory failure; insufficiency; arrest,Other respiratory disorders,Respiratory diseases
4,https://www.gofundme.com/f/tangopolly,breast cancer,DISEASE,34,47,1.0,C0006142,ENG,P,L0531189,ICD10CM,AB,C50,C50,"Malignant neoplasm of nipple and areola, right...",Breast cancer - all other types,Breast cancer,Neoplasms


In [56]:
def GetCat(df):
    u = []
    r = []
    for url in df['url'].unique():
        d = copy.deepcopy(df)
        d = d[d['url'] == url]
        cat = d.groupby(['Level 1'])['score'].sum().sort_values(ascending=False).index[0]
        u.append(url)
        r.append(cat)
    res = pd.DataFrame({'url':u, 'cat':r})
    return res

In [66]:
m[m['url'] == 'https://www.gofundme.com/f/k9v7rvss']

,url,text,label,start_char,end_char,score,cui,lang,ts,lui,sab,tty,code,ICD-10-CM CODE,ICD-10-CM CODE DESCRIPTION,Level 3,Level 2,Level 1
91,https://www.gofundme.com/f/k9v7rvss,cyst,DISEASE,36,40,0.820251,C0494710,ENG,S,L0804366,ICD10CM,PT,K09,K09,Developmental odontogenic cysts,Disorders of teeth and gingiva,Other gastrointestinal disorders,Gastrointestinal diseases


In [58]:
d = GetCat(m)

In [59]:
man = pd.read_csv('Manual_review/round 2/merged_for_python.csv')

In [60]:
m = man.merge(d, on='url')

In [61]:
m.to_csv('cui_merged.csv')

# --------
# OLD CODE
# --------

### Import json data

In [477]:
path_to_json = 'spark/comparing_heart'
json_files = ['spark/comparing_heart/' + pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

temp = pd.DataFrame()

for file in json_files:
    data = pd.read_json(file, lines=False)
    data['model'] = [file.split("/")[2].split(".")[0] for x in range(0, len(data))]
    temp = temp.append(data, ignore_index = True)


# ICD10

In [484]:
clin = temp[temp['model'] == 'ner_jsl_icd10']

clin['icd_code'] = [x.split(':::')[0] for x in clin['all_k_results']]

clin['inv_dist'] = 1/clin['distance']

clin['inv_dist'] = [x if x < 1000000000 else 100 for x in clin['inv_dist']]

merge = clin.merge(ccsr, how='left', left_on='icd_code', right_on='ICD-10-CM Code')

merge = merge[merge['Inpatient Default CCSR (Y/N/X)'] == "Y"]

merge['cat'] = [x[:3] for x in merge['CCSR Category']]

merge.groupby(['cat'])['inv_dist'].sum()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

cat
MAL    2.752546
Name: inv_dist, dtype: float64

# ATHENA

In [489]:
clin = temp[temp['model'] == 'ner_jsl_athena']

clin['inv_dist'] = 1/clin['distance']

clin['inv_dist'] = [x if x < 1000000000 else 100 for x in clin['inv_dist']]

merged = clin.merge(athena, how='left', left_on='all_k_results', right_on='concept_id')

merged = merged[merged['vocabulary_id'] == "SNOMED"]

merged['concept_code'] = merged['concept_code'].astype(int)

merge2 = merged.merge(snomed, how='left', left_on='concept_code', right_on='referencedComponentId')

merge2 = merge2[merge2['mapGroup'] == 1]
#merge2 = merge2[merge2['mapRule'] == 'OTHERWISE TRUE']

merge2['mapTarget'] = [x.replace('.','') if x != None else x for x in merge2['mapTarget']]

merge2 = merge2.dropna(subset=['mapTarget'])

merge3 = merge2.merge(ccsr, how='left', left_on='mapTarget', right_on='ICD-10-CM Code')

merge3 = merge3[merge3['Inpatient Default CCSR (Y/N/X)'] == "Y"]

merge3['cat'] = [x[:3] for x in merge3['CCSR Category']]

merge3.groupby(['cat'])['inv_dist'].sum()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


cat
MAL    1.980983
Name: inv_dist, dtype: float64

# SNOMED

In [493]:
clin = temp[temp['model'] == 'ner_jsl_snomed']

clin['inv_dist'] = 1/clin['distance']

clin['inv_dist'] = [x if x < 1000000000 else 100 for x in clin['inv_dist']]

clin['snomed_code'] = [x.split(':::')[0] for x in clin['all_k_results']]

clin['snomed_code'] = clin['snomed_code'].astype(int)

merge2 = clin.merge(snomed, how='left', left_on='snomed_code', right_on='referencedComponentId')

merge2 = merge2[merge2['mapGroup'] == 1]
#merge2 = merge2[merge2['mapRule'] == 'OTHERWISE TRUE']

merge2['mapTarget'] = [x.replace('.','') if x != None else x for x in merge2['mapTarget']]

merge2 = merge2.dropna(subset=['mapTarget'])

merge3 = merge2.merge(ccsr, how='left', left_on='mapTarget', right_on='ICD-10-CM Code')

merge3 = merge3[merge3['Inpatient Default CCSR (Y/N/X)'] == "Y"]

merge3['cat'] = [x[:3] for x in merge3['CCSR Category']]

merge3.groupby(['cat'])['inv_dist'].sum()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

cat
MAL    1.499925
Name: inv_dist, dtype: float64

In [7]:
ccsr

,ICD-10-CM CODE,ICD-10-CM CODE DESCRIPTION,Level 3,Level 2,Level 1
0,I200,Unstable angina,Coronary atherosclerosis and other heart disease,Coronary artery disease,Cardiovascular diseases
1,I201,Angina pectoris with documented spasm,Coronary atherosclerosis and other heart disease,Coronary artery disease,Cardiovascular diseases
2,I208,Other forms of angina pectoris,Coronary atherosclerosis and other heart disease,Coronary artery disease,Cardiovascular diseases
3,I209,"Angina pectoris, unspecified",Coronary atherosclerosis and other heart disease,Coronary artery disease,Cardiovascular diseases
4,I2101,ST elevation (STEMI) myocardial infarction inv...,Acute myocardial infarction,Coronary artery disease,Cardiovascular diseases
...,...,...,...,...,...
73200,Z990,Dependence on aspirator,NaN,NaN,NaN
73201,Z992,Dependence on renal dialysis,NaN,NaN,NaN
73202,Z993,Dependence on wheelchair,NaN,NaN,NaN
73203,Z9981,Dependence on supplemental oxygen,NaN,NaN,NaN


In [8]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [14]:
ccsr = pd.read_excel('Manual_Review/disease_hierarchy_11.30.2020.xlsx')

In [15]:
ccsr['ICD-10-CM CODE DESCRIPTION'] = ccsr['ICD-10-CM CODE DESCRIPTION'].str.lower()

In [72]:
process.extractOne("scoliosis", ccsr['ICD-10-CM CODE DESCRIPTION'].tolist())

('congenital scoliosis due to congenital bony malformation', 90)

In [17]:
nchar = [len(x) for x in ccsr['ICD-10-CM CODE']]

In [18]:
pd.Series(nchar).value_counts()

7    50730
6     9989
5     6710
4     5547
3      229
dtype: int64

In [20]:
[x for x in ccsr['ICD-10-CM CODE']]

['I200',
 'I201',
 'I208',
 'I209',
 'I2101',
 'I2102',
 'I2109',
 'I2111',
 'I2119',
 'I2121',
 'I2129',
 'I213',
 'I214',
 'I219',
 'I21A1',
 'I21A9',
 'I220',
 'I221',
 'I222',
 'I228',
 'I229',
 'I230',
 'I231',
 'I232',
 'I233',
 'I234',
 'I235',
 'I236',
 'I237',
 'I238',
 'I240',
 'I241',
 'I248',
 'I249',
 'I2510',
 'I25110',
 'I25111',
 'I25118',
 'I25119',
 'I252',
 'I255',
 'I256',
 'I25700',
 'I25701',
 'I25708',
 'I25709',
 'I25710',
 'I25711',
 'I25718',
 'I25719',
 'I25720',
 'I25721',
 'I25728',
 'I25729',
 'I25730',
 'I25731',
 'I25738',
 'I25739',
 'I25750',
 'I25751',
 'I25758',
 'I25759',
 'I25760',
 'I25761',
 'I25768',
 'I25769',
 'I25790',
 'I25791',
 'I25798',
 'I25799',
 'I25810',
 'I25811',
 'I25812',
 'I2589',
 'I259',
 'R072',
 'R0789',
 'R079',
 'I0981',
 'I110',
 'I130',
 'I501',
 'I5020',
 'I5021',
 'I5022',
 'I5023',
 'I5030',
 'I5031',
 'I5032',
 'I5033',
 'I5040',
 'I5041',
 'I5042',
 'I5043',
 'I50810',
 'I50811',
 'I50812',
 'I50813',
 'I50814',
 'I5